# The MovieDataBase (TMDB)

In [1]:
import pandas as pd
import numpy as np
import re
import json
from IPython.display import Markdown, display

<br>
## Data Ingestion & Data Preprocessing

In [2]:
credits = pd.read_csv("data/tmdb_5000_credits.csv")
credits_missing = credits.shape[0] - credits.dropna(how = 'any').shape[0] # .shape prende tupla, con [0] prendiamo il primo elemento che corrisponde al num di righe

# output
display(Markdown("### Credits"), 
        credits.head(), # out -> primi 5 elementi + dimensione datframe
        credits.shape)  
print("Number of missing:", credits_missing)
print("\n'movie_id' chiave primaria dataframe 'credits', contiene infatti", 
      len(credits['movie_id'].unique()), 
      "valori distinti\n")

### Credits

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


(4803, 4)

Number of missing: 0

'movie_id' chiave primaria dataframe 'credits', contiene infatti 4803 valori distinti



In [3]:
movies = pd.read_csv("data/tmdb_5000_movies.csv")
movies_missing = movies.shape[0] - movies.dropna(how = 'any').shape[0] # .shape prende tupla, con [0] prendiamo il primo elemento che corrisponde al num di righe

# output
display(Markdown("### Movies"), 
        movies.head(3), # out -> primi 5 elementi + dimensione datframe
        movies.shape)
print("Number of missing:", movies_missing)
print("\n'id' chiave primaria dataframe 'movies', contiene infatti", 
      len(movies['id'].unique()), 
      "valori distinti")

### Movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


(4803, 20)

Number of missing: 3310

'id' chiave primaria dataframe 'movies', contiene infatti 4803 valori distinti


<br>
## 1 - For each movie, compute the number of cast members

In [4]:
def n_cast(x): # per calcolare il numero di cast member
    x = json.loads(x)
    if (len(x) != 0):
        df = pd.DataFrame(x)['id'].unique() # vediamo se ci sono membri del cast ripetuti
        return len(df)
    else:
        return 0 # se non ci sono membri nel cast
    
credits['cast_members'] = credits['cast'].apply(n_cast) # creiamo nuova colonna con all'interno il numero di cast member
app = pd.DataFrame(credits['cast'].apply(json.loads)[1]) # trasformiamo in un dataframe l'elem di posizione 1 della colonna 'cast', dove abbiamo notato che eliminando i valori ripetuti, si ha un numero di elementi del cast minore. Es.Keith Richards 'id'-1430

# output
display(Markdown("<br>Es. movie_id: 285 abbiamo elementi del cast ripetuti:"),
        app[app['id']==1430], # 1430 id 'Keith Richards' - elemento ripetuto
        Markdown("<br>**Risultato (elementi del cast ripetuti non vengono conteggiati):**"),
        credits[['movie_id', 'title', 'cast_members']].head(),
        credits[['movie_id', 'title', 'cast_members']].shape,
        Markdown("<br>"))

<br>Es. movie_id: 285 abbiamo elementi del cast ripetuti:

,cast_id,character,credit_id,gender,id,name,order
12,37,Captain Teague Sparrow,52fe4232c3a36847f800b5b3,2,1430,Keith Richards,12
31,65,Captain Teague,58bc2a37c3a368663003740b,2,1430,Keith Richards,31


<br>**Risultato (elementi del cast ripetuti non vengono conteggiati):**

,movie_id,title,cast_members
0,19995,Avatar,83
1,285,Pirates of the Caribbean: At World's End,33
2,206647,Spectre,83
3,49026,The Dark Knight Rises,158
4,49529,John Carter,27


(4803, 3)

<br>

In [5]:
display(Markdown("<br>Primi 5 casi in cui cast_members == 0"), 
        credits[credits['cast_members']==0].head(), 
        credits[credits['cast_members']==0].shape,
        Markdown("**Numero di film con 0 membri, 'dovuti' ad errori di inserimento: 43**"))

<br>Primi 5 casi in cui cast_members == 0

,movie_id,title,cast,crew,cast_members
2601,17644,Barney's Great Adventure,[],"[{""credit_id"": ""52fe473b9251416c750921ff"", ""de...",0
3670,447027,Running Forever,[],[],0
3992,346081,Sardaarji,[],"[{""credit_id"": ""558ab3f4925141076f0001d7"", ""de...",0
4009,126509,2016: Obama's America,[],"[{""credit_id"": ""52fe4ae1c3a368484e16bcd7"", ""de...",0
4068,371085,Sharkskin,[],[],0


(43, 5)

**Numero di film con 0 membri, 'dovuti' ad errori di inserimento: 43**

<br>
## 2 - How many movies do not have a homepage?
Si suppone che i film che non hanno una homepage siano impostati come nulli.

In [6]:
bold = "\033[1m" # per scrivere in grassetto
reset = "\033[0;0m"

noHome = len(movies[movies['homepage'].isnull()]) # vediamo se null e prendiamo solo quelli, ovviamente usiamo len() per vedere quanti ne sono

# output
print("\n", bold, "Il numero di film che non hanno una homepage sono:", noHome, reset)
display(movies[movies['homepage'].isnull()][['id', 'title', 'homepage']].head(),
        movies[movies['homepage'].isnull()][['id', 'title', 'homepage']].shape)


  Il numero di film che non hanno una homepage sono: 3091 


,id,title,homepage
15,2454,The Chronicles of Narnia: Prince Caspian,NaN
24,254,King Kong,NaN
27,44833,Battleship,NaN
33,36668,X-Men: The Last Stand,NaN
34,62211,Monsters University,NaN


(3091, 3)

<br>
## 3 - For each year, how many movies do not have a homepage?

In [7]:
movies['year'] = pd.to_datetime(movies['release_date']).dt.year # estraggo l'anno da release_date

# movies[movies['year'].isnull()] # vediamo che c'è un anno NaN questo perchè non è indicata la release_date
# movies.loc[4553]

# prendiamo solo i film che non hanno una homepage, raggruppiamo per anno e calcoliamo quanti film sono presenti
m_noHome = movies[movies['homepage'].isnull()].groupby('year').size().reset_index().rename(columns = {0:'n_noHomepage'}) 
m_noHome.year = m_noHome.year.astype(int) # diamo valori interi agli anni (il valore NaN precedente non permetteva ciò)

# output
display(Markdown("<br>**Per ogni anno numero di film senza homepage (ordine decrescente):**"),
        m_noHome.sort_values('n_noHomepage', ascending = False).head(),
        m_noHome.shape)

<br>**Per ogni anno numero di film senza homepage (ordine decrescente):**

,year,n_noHomepage
74,2002,164
73,2001,159
86,2014,157
76,2004,149
77,2005,149


(89, 2)

<br>
## 4 - Extract the domain of each homepage

In [8]:
def multi_home(url): # per vedere se sono presenti più homepage
    url = re.split(' ', url)
    if (len(url) == 1):
        return False
    else:
        return True

multi = movies.homepage.dropna().apply(multi_home) # leviamo valori nulli altrimenti non sarebbe possibile applicare la funzione

# output
display(Markdown("<br>Film con più homepage:"),
        movies.homepage.dropna()[multi].reset_index(), # reset_index() per mostrare in formato tabellare
        Markdown("<br>"))

<br>Film con più homepage:

,index,homepage
0,3730,http://www.cargoderfilm.ch http://cargothemovi...


<br>

In [9]:
display(Markdown("<br>È possibile notare come siano presenti sia valori nulli (es. riga 3725) \
                 che doppie homepage (es. riga 3730):"),
        movies[['homepage']][3725:3735])# notiamo che ci sono valori nulli e possono essere presenti più homepage (doppia quadra per vedere come dataframe)

<br>È possibile notare come siano presenti sia valori nulli (es. riga 3725)                  che doppie homepage (es. riga 3730):

,homepage
3725,NaN
3726,NaN
3727,http://www.filmensnabbacash.se/
3728,http://www.whaleriderthemovie.com/
3729,http://www.paathefilm.com/
3730,http://www.cargoderfilm.ch http://cargothemovi...
3731,http://tv.disney.go.com/disneychannel/original...
3732,NaN
3733,NaN
3734,http://www.miramax.com/movie/the-crying-game/


In [10]:
def domain(url):
    x = []
    if (pd.isnull(url)): # se non abbiamo homepage il dominio sarà NaN
        return np.nan
    else:
        home = url.split(' ') # nel caso sia presente più di una homepage, vedere caso 3730: Cargo
        for i in range(len(home)):
            dom = re.findall(r'[\w\-+.]+', home[i]) # in dom[0] sarà presente http..., in dom[1] il dominio effettivo
            x.append(dom[1])
        return str(x)[1:-1].replace("'","") # trasformo in stringa levo parentesi quadre ([1:-1]) e apici con replace(). [2:-2] NO causa multiple homepage
    
movies['domain'] = movies['homepage'].apply(domain)

# output
display(Markdown("<br>**Dominio per ogni homepage**, con presenza di doppie homepage (3730 - Cargo) e valori nulli \
                  con la possibilità di rimuovere questi ultimi tramite l'utilizzo di *dropna()*"),
        movies[['title','homepage','domain']][3725:3735])

<br>**Dominio per ogni homepage**, con presenza di doppie homepage (3730 - Cargo) e valori nulli                   con la possibilità di rimuovere questi ultimi tramite l'utilizzo di *dropna()*

,title,homepage,domain
3725,The Sweeney,NaN,NaN
3726,Sexy Beast,NaN,NaN
3727,Easy Money,http://www.filmensnabbacash.se/,www.filmensnabbacash.se
3728,Whale Rider,http://www.whaleriderthemovie.com/,www.whaleriderthemovie.com
3729,Paa,http://www.paathefilm.com/,www.paathefilm.com
3730,Cargo,http://www.cargoderfilm.ch http://cargothemovi...,"www.cargoderfilm.ch, cargothemovie.com"
3731,High School Musical,http://tv.disney.go.com/disneychannel/original...,tv.disney.go.com
3732,Love and Death on Long Island,NaN,NaN
3733,Night Watch,NaN,NaN
3734,The Crying Game,http://www.miramax.com/movie/the-crying-game/,www.miramax.com


<br>
## 5 - Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary). There is no need to use SQL for this point

In [11]:
bold = "\033[1m"
reset = "\033[0;0m"

# vediamo quali colonne normalizzare in credits
print(bold, "CREDITS".center(80,"-"), reset)
for col in credits.columns:
    check = credits[col][0]   
    try:
        check = credits[col].apply(json.loads)
        print("Column to normalize: ", col)
    except:
        #print("Column OK: ", col)
        pass

# vediamo quali colonne normalizzare in movies
print("\n", bold, "MOVIES".center(80,"-"), reset)
for col in movies.columns:
    check = movies[col][0]   
    try:
        check = movies[col].apply(json.loads)
        print("Column to normalize: ", col)
    except:
        #print("Column OK: ", col)
        pass

 ------------------------------------CREDITS------------------------------------- 
Column to normalize:  cast
Column to normalize:  crew

  -------------------------------------MOVIES------------------------------------- 
Column to normalize:  genres
Column to normalize:  keywords
Column to normalize:  production_companies
Column to normalize:  production_countries
Column to normalize:  spoken_languages


In [12]:
def norm(col, df_start, key): # funzione per normalizzare
    col_values = df_start[col].apply(json.loads) # series che contiene in ogni riga il json decodificato (4803 righe)
    df_final = pd.DataFrame()
    for i in np.arange(len(df_start)):
        df = pd.DataFrame(col_values[i]) # per ogni lista creata: col_values[i], creiamo dataframe (estraiamo righe da json)
        df['movie_id'] = df_start[key][i] # aggiungiamo colonna 'movie_id' con il valore corrispondente
        df_final = pd.concat([df_final, df]) # concateniamo i vari datframe creati
    df_final = df_final.reset_index(drop=True) # resettiamo indice
    return df_final

In [13]:
###### NORMALIZZAZIONE #######

#credits2 = credits[['movie_id', 'cast_members']].copy() # copy() per evitare SettingWithCopyWarning
movies2 = movies[['id', 'budget', 'homepage', 'domain', 'original_language', 'original_title',
                  'overview', 'popularity', 'release_date', 'year', 'revenue', 'runtime',
                  'status', 'tagline', 'title', 'vote_average', 'vote_count']].copy() # copy() per evitare SettingWithCopyWarning
#movies2 = pd.merge(movies2, credits2, on = 'movie_id') # per appendere cast_member alla tabella movies2
movies2 = movies2.rename(index = str, columns={'id':'movie_id'}) # rinomino attributo

cast = norm("cast", credits, "movie_id")
crew = norm("crew", credits, "movie_id")

#cast['name'][0]
#for elem in cast['name']:
#    for elem2 in crew['name']:
#        if (elem==elem2):
#            print (elem)
#            break

# possono esserci stesse persone in cast e crew
#cast[cast['name']=="Stan Lee"]
#crew[crew['name']=="Stan Lee"]

actor = pd.concat([cast[['id', 'gender', 'name']], crew[['id', 'gender', 'name']]]).drop_duplicates().reset_index(drop=True) # concateniamo ed eliminiamo persone duplicate
cast = cast[['credit_id', 'cast_id', 'id', 'movie_id', 'character', 'order']] # norm
crew = crew[['credit_id', 'id', 'movie_id', 'department', 'job']] # norm

genres_id = norm("genres", movies, "id")
genres = genres_id[['id', 'name']].drop_duplicates().reset_index(drop=True)
genres_id = genres_id[['movie_id', 'id']]

keywords_id = norm("keywords", movies, "id")
keywords = keywords_id[['id', 'name']].drop_duplicates().reset_index(drop=True)
keywords_id = keywords_id[['movie_id', 'id']]

production_companies_id = norm("production_companies", movies, "id")
production_companies = production_companies_id[['id', 'name']].drop_duplicates().reset_index(drop=True)
production_companies_id = production_companies_id[['movie_id', 'id']]

production_countries_id = norm("production_countries", movies, "id")
production_countries = production_countries_id[['iso_3166_1', 'name']].drop_duplicates().reset_index(drop=True)
production_countries_id = production_countries_id[['movie_id', 'iso_3166_1']]

spoken_languages_id = norm("spoken_languages", movies, "id")
spoken_languages = spoken_languages_id[['iso_639_1', 'name']].drop_duplicates().reset_index(drop=True)
spoken_languages_id = spoken_languages_id[['movie_id', 'iso_639_1']]

In [14]:
display(Markdown("## Tabelle normalizzate:"),
        Markdown("### Movies"), movies2.head(3), movies2.shape,
        #Markdown("### Credits"), credits2.head(), credits2.shape,
        Markdown("### Cast"), cast.head(), cast.shape,
        Markdown("### Crew"), crew.head(), crew.shape,
        Markdown("### Actor"), actor.head(), actor.shape,
        Markdown("### Genres ID"), genres_id.head(), genres_id.shape,
        Markdown("### Genres"), genres.head(), genres.shape,
        Markdown("### Keywords ID"), keywords_id.head(), keywords_id.shape,
        Markdown("### Keywords"), keywords.head(), keywords.shape,
        Markdown("### Production Companies ID"), production_companies_id.head(), production_companies_id.shape,
        Markdown("### Production Companies"), production_companies.head(), production_companies.shape,
        Markdown("### Production Countries ID"), production_countries_id.head(), production_countries_id.shape,
        Markdown("### Production Countries"), production_countries.head(), production_countries.shape,
        Markdown("### Spoken Languages ID"), spoken_languages_id.head(), spoken_languages_id.shape,
        Markdown("### Spoken Languages"), spoken_languages.head(), spoken_languages.shape)

## Tabelle normalizzate:

### Movies

,movie_id,budget,homepage,domain,original_language,original_title,overview,popularity,release_date,year,revenue,runtime,status,tagline,title,vote_average,vote_count
0,19995,237000000,http://www.avatarmovie.com/,www.avatarmovie.com,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2009.0,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,285,300000000,http://disney.go.com/disneypictures/pirates/,disney.go.com,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,2007.0,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,206647,245000000,http://www.sonypictures.com/movies/spectre/,www.sonypictures.com,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,2015.0,880674609,148.0,Released,A Plan No One Escapes,Spectre,6.3,4466


(4803, 17)

### Cast

...

,movie_id,iso_639_1
0,19995,en
1,19995,es
2,285,en
3,206647,fr
4,206647,en


(6937, 2)

### Spoken Languages

,iso_639_1,name
0,en,English
1,es,Español
2,fr,Français
3,it,Italiano
4,de,Deutsch


(87, 2)

<br>
## 6 - For each movie, compute the gross margin (difference between revenue and budget)

In [15]:
movies2['gross_margin'] = movies2['revenue'] - movies2['budget']
print("\nSono presenti sia valori di budget che di revenue == 0.",
      "In particolare", movies2.budget[movies2['budget']==0].shape[0], "per il budget e",
      movies2.revenue[movies2['revenue']==0].shape[0], "per la revenue.")

# output
display(Markdown("<br>**Gross margin per film (ordine decrescente):**"),
        movies2[['title', 'revenue', 'budget', 'gross_margin']].sort_values('gross_margin', ascending = False).head(),
        movies2[['title', 'revenue', 'budget', 'gross_margin']].shape)


Sono presenti sia valori di budget che di revenue == 0. In particolare 1037 per il budget e 1427 per la revenue.


<br>**Gross margin per film (ordine decrescente):**

,title,revenue,budget,gross_margin
0,Avatar,2787965087,237000000,2550965087
25,Titanic,1845034188,200000000,1645034188
28,Jurassic World,1513528810,150000000,1363528810
44,Furious 7,1506249360,190000000,1316249360
16,The Avengers,1519557910,220000000,1299557910


(4803, 4)

<br>
## 7 - For each movie, compute the number of crew members

In [16]:
bold = "\033[1m"
reset = "\033[0;0m"

n_crew = crew[['id', 'movie_id']].drop_duplicates() # rimuoviamo persone duplicate all'interno dello stesso film (che svolgono ruoli diversi)

print("\n", bold, "Numero di film che hanno almeno un elemento in crew:", 
      len(n_crew['movie_id'].unique()), reset) # solo 4775 film hanno elementi in crew

n_crew = pd.DataFrame(n_crew.groupby('movie_id').size().reset_index().rename(columns = {0:'n_crew'}))
n_crew = pd.merge(movies2[['title', 'movie_id']], n_crew, on = 'movie_id') # mergiamo solo con le colonne title e id. Left join perchè ci sono alcuni valori nulli
n_crew = n_crew[['movie_id','title','n_crew']] # ordiniamo colonne

# output
display(Markdown("**Numero di membri della 'crew' per film (persone duplicate rimosse - ordine decrescente):**"),
        n_crew.sort_values('n_crew', ascending = False).head(), 
        n_crew.shape)


  Numero di film che hanno almeno un elemento in crew: 4775 


**Numero di membri della 'crew' per film (persone duplicate rimosse - ordine decrescente):**

,movie_id,title,n_crew
28,135397,Jurassic World,430
1133,2749,15 Minutes,329
298,106646,The Wolf of Wall Street,239
231,585,"Monsters, Inc.",232
212,435,The Day After Tomorrow,223


(4775, 3)

<br>
## 8 - For each movie, compute the number of directors

In [17]:
bold = "\033[1m"
reset = "\033[0;0m"

n_director = crew[['id', 'movie_id','job']].drop_duplicates() # notiamo che non ci sono duplicati di persone con mansioni diverse, quindi i direttori non sono duplicati per lo stesso film
n_director = n_director[n_director.job == "Director"] # vediamo se è un Director

print("\n", bold, "Numero film che hanno almeno un 'Director':", 
      len(n_director['movie_id'].unique()), reset) # solo 4773 film hanno Director

n_director = n_director.groupby('movie_id').size().reset_index().rename(columns = {0:'n_director'})
n_director = pd.merge(movies2[['title', 'movie_id']], n_director, how = 'left')#.drop('movie_id', axis=1) # mergiamo solo con le colonne title e id
n_director = n_director[['movie_id','title','n_director']]

n_director_null = n_director[n_director['n_director'].isnull()].reset_index(drop = True) # prendiamo chi non ha presente 'Director'

n_director = n_director[n_director['n_director'].isnull() == False].reset_index(drop = True)
n_director['n_director'] = n_director.n_director.astype(int)

display(Markdown("<br>**Numero di 'Director' per film (ordine decresente):**"),
        n_director.sort_values('n_director', ascending = False).head(), 
        n_director.shape,
        Markdown("**Film senza 'Director':**"),
        n_director_null.head(), 
        n_director_null.shape)


  Numero film che hanno almeno un 'Director': 4773 


<br>**Numero di 'Director' per film (ordine decresente):**

,movie_id,title,n_director
2769,2266,"Paris, je t'aime",21
3053,87818,Movie 43,12
4046,756,Fantasia,12
465,49948,Fantasia 2000,8
4426,3170,Bambi,7


(4773, 3)

**Film senza 'Director':**

,movie_id,title,n_director
0,19615,Flying By,NaN
1,447027,Running Forever,NaN
2,26379,Paa,NaN
3,55831,Boynton Beach Club,NaN
4,371085,Sharkskin,NaN


(30, 3)

<br>
## 9 - For each language, compute the number of movies where such language is spoken

In [18]:
n_language = pd.merge(spoken_languages_id, spoken_languages, on = 'iso_639_1')
n_language = n_language.groupby(['name', 'iso_639_1']).size().reset_index().rename(columns = {0:'n_language'})

display(Markdown("<br>**Per ogni lingua, numero di film in cui viene parlata (ordine decrescente):**"),
        n_language[['iso_639_1', 'name', 'n_language']].sort_values('n_language', ascending = False).head(),
        n_language.shape)

<br>**Per ogni lingua, numero di film in cui viene parlata (ordine decrescente):**

,iso_639_1,name,n_language
36,en,English,4485
39,fr,Français,437
37,es,Español,351
34,de,Deutsch,262
43,it,Italiano,188


(87, 3)

<br>
## 10 - For each company and each decade, compute the overall revenue

In [19]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) # no scientific notation

movies2['decade'] = movies2.year//10*10 # calcolo decade

decade_false = movies2[movies2['decade'].isnull()] # vediamo dove decade e quindi year è nullo

# merge delle tabelle di interesse
overall = pd.merge(movies2, production_companies_id, on = 'movie_id')
overall = pd.merge(overall, production_companies, on = 'id')

# appoggio per la richiesta 12
overall2 = overall[['decade', 'name', 'title', 'revenue']].copy() # appoggio per la richiesta 12
overall2.decade = overall2.decade.astype(int)

# calcolo overall revenue
overall = overall.groupby(['name', 'decade']).revenue.sum().reset_index() # sommiamo revenue se stesso name e decade
overall.decade = overall.decade.astype(int)

# nuovo overall revenue calcolato come rapporto tra revenue e numero di aziende che hanno collaborato alla realizzazione di un film
number_comp_movie = production_companies_id.groupby('movie_id').count().rename(columns = {'id':'n_comp'}).reset_index()
overall_div = pd.merge(number_comp_movie, movies2, on = 'movie_id')
overall_div['final_revenue'] = overall_div['revenue']/overall_div['n_comp']
overall_div = pd.merge(overall_div, production_companies_id, on = 'movie_id')
overall_div = pd.merge(overall_div, production_companies, on = 'id')
overall_div = overall_div.groupby(['name', 'decade']).final_revenue.sum().reset_index() # sommiamo revenue se stesso name e decade
overall_div.decade = overall_div.decade.astype(int)

# output
display(Markdown("<br>Film di cui non è possibile avere decade (valore nullo):"),
        decade_false[['title', 'release_date']],
        Markdown("<br>**Revenue totale calcolato per ogni azienda e decade (ordine decrescente):**"),
        overall.sort_values('revenue', ascending = False).head(), 
        overall.shape,
        Markdown("<br>**Revenue finale calcolato per ogni azienda e decade (rapporto tra revenue e numero di aziende \
        che hanno collaborato alla realizzazione del film - ordine decrescente):**"),
        overall_div.sort_values('final_revenue', ascending = False).head(),
        overall_div.shape)

<br>Film di cui non è possibile avere decade (valore nullo):

,title,release_date
4553,America Is Still the Place,NaN


<br>**Revenue totale calcolato per ogni azienda e decade (ordine decrescente):**

,name,decade,revenue
5881,Warner Bros.,2000,22897419289
5604,Twentieth Century Fox Film Corporation,2000,15889321197
5692,Universal Pictures,2000,15725218820
5693,Universal Pictures,2010,15521006459
4082,Paramount Pictures,2010,15110115837


(6123, 3)

<br>**Revenue finale calcolato per ogni azienda e decade (rapporto tra revenue e numero di aziende         che hanno collaborato alla realizzazione del film - ordine decrescente):**

,name,decade,final_revenue
3381,Marvel Studios,2010,6190159565.25
5881,Warner Bros.,2000,6133589457.85
5856,Walt Disney Pictures,2010,6003551354.92
1145,Columbia Pictures,2010,5121072773.12
5693,Universal Pictures,2010,4896168347.62


(6123, 3)

***Non sapendo quale soluzione sia la più consona, d'ora in poi verrà considerato come overall revenue il primo mostrato, ovvero quello senza i rapporti con il numero di aziende che hanno collaborato alla realizzazione del film***

<br>
## 11 - For each decade, compute the company with maximum revenue

*Nel primo blocco di codice verranno calcolate le aziende, per ogni decade, con il revenue massimo (revenue sommato in base a tutti i film prodotti in quella decade).*

*Nel secondo blocco di codice verranno calcolate, per ogni decade, le aziende con revenue massimo prendendo in considerazione il valore per un singolo film.*

In [20]:
max_rev = overall.groupby('decade')['revenue'].max().reset_index() # vediamo massimo per ogni decade (ce ne può essere più di uno per decade)
max_rev = pd.merge(max_rev, overall, on = ['decade', 'revenue'])[['decade', 'name', 'revenue']] # mergiamo per trovare tutte le aziende con lo stesso max per decade

# output
display(Markdown("<br>Notiamo che ci possono essere più aziende con lo stesso revenue massimo per decade (un film può avere più aziende che hanno collaborato alla creazione). Es:"), 
        overall.sort_values(['decade', 'revenue'], ascending = False).tail(2),
        Markdown("<br>**Per ogni decade, aziende con revenue massimo**"),
        max_rev,
        max_rev.shape)

<br>Notiamo che ci possono essere più aziende con lo stesso revenue massimo per decade (un film può avere più aziende che hanno collaborato alla creazione). Es:

,name,decade,revenue
5555,Triangle Film Corporation,1910,8394751
5872,Wark Producing Corp.,1910,8394751


<br>**Per ogni decade, aziende con revenue massimo**

,decade,name,revenue
0,1910,Triangle Film Corporation,8394751
1,1910,Wark Producing Corp.,8394751
2,1920,Metro-Goldwyn-Mayer (MGM),26358000
3,1930,Metro-Goldwyn-Mayer (MGM),400176459
4,1930,Selznick International Pictures,400176459
5,1940,Walt Disney Productions,351747150
6,1950,Metro-Goldwyn-Mayer (MGM),73770324
7,1960,Eon Productions,598134765
8,1970,United Artists,1392271577
9,1980,Paramount Pictures,4431144486


(13, 3)

In [21]:
max_rev2 = movies2.groupby('decade')['revenue'].max().reset_index() # ora raggruppiamo da movies per considerare il singolo film
max_rev2 = pd.merge(max_rev2, overall2, on = ['decade', 'revenue'])[['decade', 'name', 'title', 'revenue']]

# output
display(Markdown("<br>**Per ogni decade aziende con 'revenue' massimo prendendo in considerazione un solo \
                 film (per uno stesso film possono esserci più aziende):**"),
        max_rev2,
        max_rev2.shape)

<br>**Per ogni decade aziende con 'revenue' massimo prendendo in considerazione un solo                  film (per uno stesso film possono esserci più aziende):**

,decade,name,title,revenue
0,1910.00,Triangle Film Corporation,Intolerance,8394751
1,1910.00,Wark Producing Corp.,Intolerance,8394751
2,1920.00,Metro-Goldwyn-Mayer (MGM),The Big Parade,22000000
3,1930.00,Metro-Goldwyn-Mayer (MGM),Gone with the Wind,400176459
4,1930.00,Selznick International Pictures,Gone with the Wind,400176459
5,1940.00,Walt Disney Productions,Bambi,267447150
6,1950.00,Twentieth Century Fox Film Corporation,The Robe,36000000
7,1950.00,Paramount Pictures,The Greatest Show on Earth,36000000
8,1960.00,Twentieth Century Fox Film Corporation,The Sound of Music,286214286
9,1960.00,Robert Wise Productions,The Sound of Music,286214286


(24, 4)

<br>
## 12 - In each year, how many movies have revenue smaller than the budget?

In [22]:
movies_smaller = movies2.loc[movies2['gross_margin']<0, ['year', 'gross_margin']]
movies_smaller.year = movies_smaller.year.astype(int)
movies_smaller = movies_smaller.groupby('year').size().reset_index().rename(columns = {0:'n_neg_gross_margin'})

# output
display(Markdown("<br>**Numero di film che hanno una 'revenue' minore del budget a disposizione (ordine decrescente):**"),
        movies_smaller.sort_values(['n_neg_gross_margin'],  ascending = False).head(),
        movies_smaller.shape)

<br>**Numero di film che hanno una 'revenue' minore del budget a disposizione (ordine decrescente):**

,year,n_neg_gross_margin
47,2009,78
40,2002,71
49,2011,70
53,2015,67
48,2010,65


(55, 2)